In [28]:
from pathlib import Path
from collections import Counter

# ====== 경로 설정 ======
#LABEL_DIR = Path("/home/dw/ws_job_msislab/Golf_Project/data/20250721/20250721_good_data")
#LABEL_DIR = Path("/home/dw/ws_job_msislab/Golf_Project/data/20250725/20250725_good_data")
LABEL_DIR = Path("/home/dw/ws_job_msislab/Golf_Project/data/20250904/20250904_good_data")

# YOLO 클래스 이름 매핑
classes = [
    "Divot",         # 0
    "Fixed_Divot",   # 1
    "Diseased_Grass",# 2
    "Confused_Object",# 3
    "Pole",          # 4
    "Sprinkler",     # 5
    "Drain",         # 6
    "Golf ball"      # 7
]

counter = Counter()

for txt_file in LABEL_DIR.glob("*.txt"):
    if txt_file.name == "classes.txt":     # ★ classes.txt는 건너뛰기
        continue
    with open(txt_file, "r") as f:
        for line in f:
            parts = line.strip().split()
            if parts:
                cls_id = int(parts[0])
                counter[cls_id] += 1

# ====== 결과 출력 ======
print("=== 클래스별 객체 개수 ===")
for i, cls_name in enumerate(classes):
    print(f"{cls_name:<15}: {counter.get(i, 0)}")

print("\n총 객체 수:", sum(counter.values()))


=== 클래스별 객체 개수 ===
Divot          : 2424
Fixed_Divot    : 3677
Diseased_Grass : 389
Confused_Object: 0
Pole           : 106
Sprinkler      : 20
Drain          : 162
Golf ball      : 157

총 객체 수: 6935


In [32]:
# === N개 경로의 YOLO 데이터 머지(복사) + 매칭/충돌 리포트 =====================
from pathlib import Path
import shutil
from collections import defaultdict, Counter

# ===== 설정: 여러 소스 폴더 + 목적지 폴더 =====
SOURCES = [
    Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20250721_good_data"),
    Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20250725_good_data"),
    Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20250904_good_data"),
    # Path("/home/dw/ws_job_msislab/Golf_Project/data/20250725/20250725_good_data"),
]
DST  = Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20250912_merge_data")

IMG_EXTS_PRIORITY = [".jpg", ".jpeg", ".png", ".bmp", ".webp", ".tif", ".tiff"]
IMG_EXTS = set(IMG_EXTS_PRIORITY)
LABEL_EXT = ".txt"

# ===== 유틸 =====
def is_image(p: Path) -> bool:
    return p.suffix.lower() in IMG_EXTS

def is_label(p: Path) -> bool:
    return p.suffix.lower() == LABEL_EXT and p.name != "classes.txt"  # classes.txt 제외

def best_image_choice(paths):
    """동일 stem의 다중 이미지가 있다면 우선순위 확장자 기준으로 1개 선택"""
    if len(paths) == 1:
        return paths[0]
    def keyfunc(p: Path):
        try:
            return IMG_EXTS_PRIORITY.index(p.suffix.lower())
        except ValueError:
            return len(IMG_EXTS_PRIORITY)
    return sorted(paths, key=keyfunc)[0]

def collect_items(src_dir: Path):
    """src_dir 재귀 탐색 → [{stem, img, lbl, src}]"""
    images_by_stem = defaultdict(list)
    labels_by_stem = {}
    for p in src_dir.rglob("*"):
        if not p.is_file():
            continue
        if is_image(p):
            images_by_stem[p.stem].append(p)
        elif is_label(p):
            labels_by_stem.setdefault(p.stem, p)  # 동일 stem 다수 라벨이면 첫 번째만 사용

    items = []
    all_stems = set(images_by_stem.keys()) | set(labels_by_stem.keys())
    for s in sorted(all_stems):
        img_paths = images_by_stem.get(s, [])
        img = best_image_choice(img_paths) if img_paths else None
        lbl = labels_by_stem.get(s)
        items.append({"stem": s, "img": img, "lbl": lbl, "src": src_dir})
    return items

def next_available_stem(dst_dir: Path, base_stem: str, need_img_ext: str|None, need_lbl: bool):
    """DST 내에서 충돌 없도록 stem_1, stem_2 ... 생성"""
    cand = base_stem
    i = 1
    while True:
        img_ok = True
        lbl_ok = True
        if need_img_ext:
            img_ok = not (dst_dir / f"{cand}{need_img_ext}").exists()
        if need_lbl:
            lbl_ok = not (dst_dir / f"{cand}{LABEL_EXT}").exists()
        if img_ok and lbl_ok:
            return cand
        i += 1
        cand = f"{base_stem}_{i}"

def copy_safe(src: Path, dst: Path):
    dst.parent.mkdir(parents=True, exist_ok=True)
    shutil.copy2(str(src), str(dst))

def show_samples(title, arr, maxn=8):
    print(f"\n[{title}] (최대 {maxn}개 예시)")
    for s in arr[:maxn]:
        print(" -", s)

# ===== 수집 & 복사 =====
DST.mkdir(parents=True, exist_ok=True)

overall_counts = Counter()
renamed = []   # (old_stem -> new_stem, src)
name_map_samples = []

per_source_counts = {}

for SRC in SOURCES:
    items = collect_items(SRC)
    counts = Counter()
    for it in items:
        stem, img, lbl = it["stem"], it["img"], it["lbl"]
        need_img = img is not None
        need_lbl = lbl is not None

        if need_img and need_lbl:
            counts["pairs"] += 1
        elif need_img:
            counts["img_only"] += 1
        elif need_lbl:
            counts["lbl_only"] += 1

        img_ext = img.suffix.lower() if img else None

        # 충돌 방지용 stem 선택 (이미지/라벨 모두 고려)
        dest_stem = next_available_stem(DST, stem, img_ext, need_lbl)

        # 리네임 여부 기록
        if dest_stem != stem:
            renamed.append((stem, dest_stem, it["src"]))
            if len(name_map_samples) < 8:
                name_map_samples.append(f"{it['src'].name}: {stem}  →  {dest_stem}")

        # 실제 복사
        if need_img:
            dst_img = DST / f"{dest_stem}{img_ext}"
            copy_safe(img, dst_img)
            counts["copied_imgs"] += 1
        if need_lbl:
            dst_lbl = DST / f"{dest_stem}{LABEL_EXT}"
            copy_safe(lbl, dst_lbl)
            counts["copied_lbls"] += 1

    per_source_counts[str(SRC)] = counts
    overall_counts.update(counts)
    overall_counts["stems"] += len(items)

# ===== 리포트 =====
print("====================================")
print("[MERGE RESULT]")
print(f"- DST : {DST}")
print("------------------------------------")
print("[Per-Source Summary]")
for src, c in per_source_counts.items():
    print(f"* {src}")
    print(f"  - 짝(이미지+라벨) : {c.get('pairs', 0)}")
    print(f"  - 이미지만 있음   : {c.get('img_only', 0)}")
    print(f"  - 라벨만 있음     : {c.get('lbl_only', 0)}")
    print(f"  - 복사된 이미지   : {c.get('copied_imgs', 0)}")
    print(f"  - 복사된 라벨     : {c.get('copied_lbls', 0)}")

print("------------------------------------")
print("[Overall]")
print(f"- 총 stem 개수           : {overall_counts.get('stems', 0)}")
print(f"- 짝(이미지+라벨)         : {overall_counts.get('pairs', 0)}")
print(f"- 이미지만 있음           : {overall_counts.get('img_only', 0)}")
print(f"- 라벨만 있음             : {overall_counts.get('lbl_only', 0)}")
print("------------------------------------")
print(f"- 복사된 이미지 수        : {overall_counts.get('copied_imgs', 0)}")
print(f"- 복사된 라벨(.txt) 수    : {overall_counts.get('copied_lbls', 0)}")
print(f"- 이름 충돌로 리네임된 건 : {len(renamed)}")
print("====================================")

show_samples("리네임 매핑 예시 (old → new)", name_map_samples)


[MERGE RESULT]
- DST : /home/dw/ws_job_msislab/Golf_Project/data/for_study/20250912_merge_data
------------------------------------
[Per-Source Summary]
* /home/dw/ws_job_msislab/Golf_Project/data/for_study/20250721_good_data
  - 짝(이미지+라벨) : 1760
  - 이미지만 있음   : 68
  - 라벨만 있음     : 0
  - 복사된 이미지   : 1828
  - 복사된 라벨     : 1760
* /home/dw/ws_job_msislab/Golf_Project/data/for_study/20250725_good_data
  - 짝(이미지+라벨) : 462
  - 이미지만 있음   : 25
  - 라벨만 있음     : 0
  - 복사된 이미지   : 487
  - 복사된 라벨     : 462
* /home/dw/ws_job_msislab/Golf_Project/data/for_study/20250904_good_data
  - 짝(이미지+라벨) : 4306
  - 이미지만 있음   : 29
  - 라벨만 있음     : 1
  - 복사된 이미지   : 4335
  - 복사된 라벨     : 4307
------------------------------------
[Overall]
- 총 stem 개수           : 6651
- 짝(이미지+라벨)         : 6528
- 이미지만 있음           : 122
- 라벨만 있음             : 1
------------------------------------
- 복사된 이미지 수        : 6650
- 복사된 라벨(.txt) 수    : 6529
- 이름 충돌로 리네임된 건 : 0

[리네임 매핑 예시 (old → new)] (최대 8개 예시)


In [33]:
from pathlib import Path
from collections import Counter

# ====== 경로 설정 ======
LABEL_DIR = Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20250912_merge_data")

# YOLO 클래스 이름 매핑
classes = [
    "Divot",        # 0
    "Fixed_Divot",  # 1
    "Diseased_Grass",# 2
    "Confused_Object",         # 3
    "Pole",     # 4
    "Sprinkler",    # 5
    "Drain",        # 6
    "Golf ball"     # 7
]

# ====== 객체 카운트 ======
counter = Counter()

# txt 라벨 파일 순회
for txt_file in LABEL_DIR.glob("*.txt"):
    with open(txt_file, "r") as f:
        for line in f:
            parts = line.strip().split()
            if not parts:
                continue
            cls_id = int(parts[0])
            counter[cls_id] += 1

# ====== 결과 출력 ======
print("=== 클래스별 객체 개수 ===")
for i, cls_name in enumerate(classes):
    print(f"{cls_name:<15}: {counter.get(i, 0)}")

print("\n총 객체 수:", sum(counter.values()))

=== 클래스별 객체 개수 ===
Divot          : 7668
Fixed_Divot    : 8045
Diseased_Grass : 1151
Confused_Object: 42
Pole           : 366
Sprinkler      : 141
Drain          : 425
Golf ball      : 471

총 객체 수: 18309


In [31]:
# augment_balance_final_custom.py
# - 사용자가 원하는 TARGET(클래스별 최종 객체 수)을 직접 설정
# - 재사용 제한을 클래스별로 가산할 수 있도록 개선
# - 나머지 안전장치(yolo_sanitize, 빈/이상 라인 제거, 로그 등) 유지

from pathlib import Path
from collections import Counter, defaultdict
import random, csv
import cv2
import albumentations as A

# ========== 사용자 설정 ==========
BASE = Path("/home/dw/ws_job_msislab/Golf_Project/data/20250904/20250904_good_data")
OUT_DIR = BASE / "aug_balanced"     # 단일 폴더 (이미지/라벨 저장)
IMG_EXTS = [".jpg", ".jpeg", ".png"]
SEED = 42
random.seed(SEED)

# (필수) 최종 목표 개수를 직접 지정하세요. (원본처럼!!)
# 예시: Divot 3500, Fixed_Divot 3500, Diseased_Grass 300, 나머지는 기존 유지 or 적절히 설정
# 클래스 매핑: 0 Divot, 1 Fixed_Divot, 2 Diseased_Grass, 3 Pole, 4 Obstacle, 5 Sprinkler, 6 Drain, 7 Golf ball
TARGET = {
    0: 5000, # Divot
    1: 5000, # Fixed_Divot
    2: 1000,  # Diseased_Grass
    3: 0,   # Confused_Object (예시)
    4: 330,    # Pole (예시)
    5: 80,   # Sprinkler (예시)
    6: 400,   # Drain (예시)
    7: 400,   # Golf ball (예시)
}
ALL_CLASS_IDS = list(range(8))
ALLOW_IDS = set(ALL_CLASS_IDS)

# 재사용 제한 정책
# - 기본 회수
MAX_USES_BASE = 2
# - 클래스별 가산치(해당 클래스를 포함하고 "아직 부족"한 경우에만 적용)
#   예: 2와 3이 특히 부족하면 2회 더 허용, 5는 1회 더 허용… 필요에 맞게 수정
MAX_USES_BOOST_PER_CLASS = {
    0: 5,
    1: 5,
    2: 5,   # Dieased_Grass 부족하면 +2
    3: 5,
    4: 5,
    5: 5,   # Pole 부족하면 +2
    6: 5,   # Sprinkler 부족하면 +1
    7: 5,
    # 필요시 0..7 아무거나 추가/수정
}

# 배경(라벨 없는 이미지) 증강 배수
# 값=2 → 원본 1 + 증강 2 = 총 3배
BG_AUG_MULTIPLIER = 2

# 증강 파이프라인(색/조명 중심, 강한 광량 변동 + 약한 감마)
aug = A.Compose([

    # 1. 좌우 반전: 50% 확률
    A.HorizontalFlip(p=0.5),

    # 2. 밝기/대비 랜덤 조절: 90% 확률
    #    brightness_limit을 -0.30 ~ +0.30으로 확대 → 광량(밝기) 변화를 크게
    #    contrast_limit은 기존 유지
    A.RandomBrightnessContrast(
        brightness_limit=(-0.15, 0.15),
        contrast_limit=(-0.08, 0.08),
        p=0.9
    ),

    # 3. 색조/채도/밝기 변형: 90% 확률
    A.HueSaturationValue(
        hue_shift_limit=6,
        sat_shift_limit=8,
        val_shift_limit=8,
        p=0.9
    ),

    # 4. RGB 채널별 색상 이동: 60% 확률
    A.RGBShift(
        r_shift_limit=8,
        g_shift_limit=8,
        b_shift_limit=8,
        p=0.6
    ),

    # 5. 감마 변환: 40% 확률
    #    감마 변화 폭을 줄여 전체 톤 커브 변형은 최소화 (0.95~1.1)
    A.RandomGamma(
        gamma_limit=(95, 105),   # 기존 (80,140) → 좁게 (95,110)
        p=0.3
    ),

    # 6. 로컬 콘트라스트 또는 노이즈: 45% 확률
    A.OneOf([
        A.CLAHE(clip_limit=3.0, tile_grid_size=(8, 8), p=0.6),
        A.ISONoise(color_shift=(0.01, 0.08), intensity=(0.08, 0.35), p=0.4),
    ], p=0.45),

    # 7. 아주 가끔 저채도/그레이 톤 변환 → 텍스처 대비 학습
    # A.ToGray(p=0.05),

    # 8. 미세한 블러/샤픈 중 하나: 25% 확률
    A.OneOf([
        A.MotionBlur(blur_limit=3, p=0.4),
        A.GaussianBlur(blur_limit=3, p=0.4),
        A.Sharpen(alpha=(0.05, 0.2), lightness=(0.9, 1.1), p=0.2),
    ], p=0.20),

], bbox_params=A.BboxParams(
    format="yolo",                
    label_fields=["class_labels"],
    min_visibility=0.25           
))


# ========== 유틸 ==========
def ensure_out_dir():
    OUT_DIR.mkdir(parents=True, exist_ok=True)

def read_yolo_label(lbl: Path):
    boxes, cls = [], []
    if not lbl.exists():
        return boxes, cls
    with open(lbl, "r") as f:
        for line in f:
            p = line.strip().split()
            if len(p) < 5:
                continue
            try:
                c = int(float(p[0]))
            except Exception:
                continue
            try:
                x, y, w, h = map(float, p[1:5])
            except Exception:
                continue
            x = min(max(x, 0.0), 1.0)
            y = min(max(y, 0.0), 1.0)
            w = min(max(w, 0.0), 1.0)
            h = min(max(h, 0.0), 1.0)
            if w <= 0.0 or h <= 0.0:
                continue
            boxes.append([x, y, w, h]); cls.append(c)
    return boxes, cls

def write_yolo_label(lbl_path: Path, boxes, cls, *, allow_ids=ALLOW_IDS):
    lbl_path.parent.mkdir(parents=True, exist_ok=True)
    with open(lbl_path, "w") as f:
        for c, (x, y, w, h) in zip(cls, boxes):
            try:
                c_int = int(float(c))
            except Exception:
                continue
            if allow_ids is not None and c_int not in allow_ids:
                continue
            x = float(max(0.0, min(1.0, x)))
            y = float(max(0.0, min(1.0, y)))
            w = float(max(0.0, min(1.0, w)))
            h = float(max(0.0, min(1.0, h)))
            if w <= 0.0 or h <= 0.0:
                continue
            f.write(f"{c_int} {x:.6f} {y:.6f} {w:.6f} {h:.6f}\n")

def yolo_sanitize(boxes):
    eps = 1e-6
    out = []
    for x, y, w, h in boxes:
        l = x - w/2.0; r = x + w/2.0
        t = y - h/2.0; b = y + h/2.0
        l = max(0.0, min(1.0, l))
        r = max(0.0, min(1.0, r))
        t = max(0.0, min(1.0, t))
        b = max(0.0, min(1.0, b))
        if r - l <= eps or b - t <= eps:
            continue
        nx = (l + r) / 2.0
        ny = (t + b) / 2.0
        nw = (r - l)
        nh = (b - t)
        nx = min(max(nx, eps), 1.0 - eps)
        ny = min(max(ny, eps), 1.0 - eps)
        nw = min(max(nw, eps), 1.0 - eps)
        nh = min(max(nh, eps), 1.0 - eps)
        out.append([nx, ny, nw, nh])
    return out

def find_image_for_label(lbl: Path):
    stem = lbl.stem
    for ext in IMG_EXTS:
        p = lbl.with_suffix(ext)
        if p.exists(): return p
    for parent in [lbl.parent, *lbl.parents]:
        img_dir = parent / "images"
        if img_dir.exists():
            for ext in IMG_EXTS:
                p = img_dir / f"{stem}{ext}"
                if p.exists(): return p
    for ext in IMG_EXTS:
        hits = list(BASE.rglob(f"{stem}{ext}"))
        if hits: return hits[0]
    return None

def current_counts(root: Path):
    cnt = Counter()
    for lbl in root.rglob("*.txt"):
        _, cls = read_yolo_label(lbl)
        cnt.update(cls)
    return cnt

def collect_background_images(base: Path):
    label_stems = {p.stem for p in base.rglob("*.txt")}
    bg_images = []
    for ext in IMG_EXTS:
        for img in base.rglob(f"*{ext}"):
            if img.stem not in label_stems:
                bg_images.append(img)
    return sorted(bg_images)

def load_image(path: Path):
    img = cv2.imread(str(path))
    if img is None:
        raise RuntimeError(f"Failed to read image: {path}")
    return img

def next_unique_name(stem: str, used: set, tag: str):
    i = 1
    while True:
        s = f"{stem}_{tag}{i:04d}"
        if s not in used:
            used.add(s)
            return s
        i += 1

def effective_cap_for_file(file_cls_counter: Counter, deficit: Counter):
    """
    이 파일(라벨)의 클래스 구성과 현재 부족분을 보고
    MAX_USES_BASE + (부족 클래스 중 최대 가산치) 를 반환.
    """
    cap = MAX_USES_BASE
    best_boost = 0
    for c, n in file_cls_counter.items():
        if n <= 0: 
            continue
        if deficit.get(c, 0) > 0:
            boost = MAX_USES_BOOST_PER_CLASS.get(c, 0)
            if boost > best_boost:
                best_boost = boost
    return cap + best_boost

# ========== 실행 ==========
ensure_out_dir()

# 1) 배경 3배(원본 + 증강2)
bg_list = collect_background_images(BASE)
print(f"Background images: {len(bg_list)}")

used_names = set()
for bg_img in bg_list:
    try:
        img = load_image(bg_img)
    except Exception:
        continue
    for _ in range(BG_AUG_MULTIPLIER):
        out_stem = next_unique_name(bg_img.stem, used_names, tag="bg")
        out_img = OUT_DIR / f"{out_stem}{bg_img.suffix.lower()}"
        out_lbl = OUT_DIR / f"{out_stem}.txt"
        transformed = aug(image=img, bboxes=[], class_labels=[])
        aug_img = transformed["image"]
        cv2.imwrite(str(out_img), aug_img)
        with open(out_lbl, "w") as f:
            pass
print("Background augmentation done.")

# 2) 라벨 쌍 후보
label_files = sorted(BASE.rglob("*.txt"))
candidates = []  # (img_path, lbl_path, file_counts, stem)
skipped = 0
for lbl in label_files:
    boxes, cls_list = read_yolo_label(lbl)
    if not cls_list:
        continue
    img = find_image_for_label(lbl)
    if img is None or not img.exists():
        skipped += 1
        continue
    fcnt = Counter(cls_list)
    candidates.append((img, lbl, fcnt, lbl.stem))
if not candidates:
    raise SystemExit("No labeled image-label pairs found.")
print(f"Labeled pairs: {len(candidates)}, skipped(no image match): {skipped}")

# 3) 초기 카운트(원본)
base_cnt = current_counts(BASE)
print("Base counts:", dict(base_cnt))

# 4) 타깃/결손 세팅
cur = Counter(base_cnt)
# TARGET이 현재보다 작은 경우는 결손 0으로 간주(감소 작업은 하지 않음)
deficit = Counter({c: max(0, TARGET.get(c, 0) - cur.get(c, 0)) for c in ALL_CLASS_IDS})

use_count = defaultdict(int)
log_path = OUT_DIR / "log.csv"
with open(log_path, "w", newline="") as f:
    w = csv.writer(f)
    w.writerow(["src_img","src_lbl","out_img","out_lbl","add_per_class","totals_after"])

MAX_ITERS = 200000
iters = 0

def score(deficit: Counter, fcnt: Counter):
    # 남은 결손에 기여도가 큰 파일 우선
    return sum(deficit[c] * fcnt.get(c, 0) for c in ALL_CLASS_IDS if deficit[c] > 0)

while any(deficit[c] > 0 for c in ALL_CLASS_IDS) and iters < MAX_ITERS:
    iters += 1
    best = None
    best_score = 0

    for img, lbl, fcnt, stem in candidates:
        cap = effective_cap_for_file(fcnt, deficit)
        if use_count[img] >= cap:
            continue
        sc = score(deficit, fcnt)
        if sc > best_score:
            best_score = sc
            best = (img, lbl, fcnt, stem)

    if not best or best_score <= 0:
        print("No more useful candidates for remaining deficits. Stopping.")
        break

    img_path, lbl_path, fcnt, stem = best
    out_stem = next_unique_name(stem, used_names, tag="dup")
    out_img = OUT_DIR / f"{out_stem}{img_path.suffix.lower()}"
    out_lbl = OUT_DIR / f"{out_stem}.txt"

    # 증강
    img = load_image(img_path)
    boxes0, cls0 = read_yolo_label(lbl_path)
    boxes0 = yolo_sanitize(boxes0)

    transformed = aug(image=img, bboxes=boxes0, class_labels=cls0)
    aug_img = transformed["image"]
    aug_boxes = transformed["bboxes"]
    aug_cls = transformed["class_labels"]

    keep_boxes, keep_cls = [], []
    for (x, y, w, h), c in zip(aug_boxes, aug_cls):
        if w > 0 and h > 0:
            keep_boxes.append([float(x), float(y), float(w), float(h)])
            keep_cls.append(int(float(c)))

    # 실제로 부족 클래스에 기여하는지 확인
    test_cnt = Counter(keep_cls)
    if sum(test_cnt[c] for c in ALL_CLASS_IDS if deficit[c] > 0) <= 0 or not keep_cls:
        use_count[img_path] += 1
        continue

    cv2.imwrite(str(out_img), aug_img)
    write_yolo_label(out_lbl, keep_boxes, keep_cls, allow_ids=ALLOW_IDS)

    _, saved_cls = read_yolo_label(out_lbl)
    add_cnt = Counter(saved_cls)

    cur.update(add_cnt)
    for c in ALL_CLASS_IDS:
        deficit[c] = max(0, TARGET.get(c, 0) - cur.get(c, 0))
    use_count[img_path] += 1

    add_str = "{" + ", ".join(f"{k}:{add_cnt.get(k,0)}" for k in ALL_CLASS_IDS) + "}"
    tot_str = "{" + ", ".join(f"{k}:{cur.get(k,0)}" for k in ALL_CLASS_IDS) + "}"
    with open(log_path, "a", newline="") as f:
        w = csv.writer(f)
        w.writerow([str(img_path), str(lbl_path), str(out_img), str(out_lbl), add_str, tot_str])

    if iters % 50 == 0 or all(deficit[c]==0 for c in ALL_CLASS_IDS):
        print(f"[{iters}] cur={dict(cur)} deficit={dict(deficit)} last={out_stem}")

print("\n=== Finished ===")
print("Final counts:", dict(cur))
print("Deficit:", dict(deficit))
print(f"Saved to: {OUT_DIR}")
print(f"Log: {log_path}")

Background images: 28
Background augmentation done.
Labeled pairs: 2898, skipped(no image match): 0
Base counts: {0: 2424, 1: 3676, 2: 389, 6: 162, 7: 157, 5: 20, 4: 106}
[50] cur={0: 2677, 1: 3840, 2: 389, 6: 162, 7: 157, 5: 20, 4: 106} deficit={0: 2323, 1: 1160, 2: 611, 3: 0, 4: 224, 5: 60, 6: 238, 7: 243} last=20250904_04342_dup0004
[100] cur={0: 2908, 1: 3962, 2: 389, 6: 162, 7: 157, 5: 20, 4: 106} deficit={0: 2092, 1: 1038, 2: 611, 3: 0, 4: 224, 5: 60, 6: 238, 7: 243} last=20250904_06848_dup0002
[150] cur={0: 3073, 1: 4179, 2: 389, 6: 162, 7: 157, 5: 20, 4: 106} deficit={0: 1927, 1: 821, 2: 611, 3: 0, 4: 224, 5: 60, 6: 238, 7: 243} last=20250904_04332_dup0003
[200] cur={0: 3273, 1: 4279, 2: 389, 6: 162, 7: 157, 5: 20, 4: 106} deficit={0: 1727, 1: 721, 2: 611, 3: 0, 4: 224, 5: 60, 6: 238, 7: 243} last=20250904_06854_dup0004
[250] cur={0: 3447, 1: 4431, 2: 389, 6: 162, 7: 157, 5: 20, 4: 106} deficit={0: 1553, 1: 569, 2: 611, 3: 0, 4: 224, 5: 60, 6: 238, 7: 243} last=20250904_04346_d

In [37]:
from pathlib import Path
from collections import Counter

# ====== 경로 설정 ======
LABEL_DIR = Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20250912_merge_data_split")

# YOLO 클래스 이름 매핑
classes = [
    "Divot",        # 0
    "Fixed_Divot",  # 1
    "Diseased_Grass",# 2
    "Confused_Object",         # 3
    "Pole",     # 4
    "Sprinkler",    # 5
    "Drain",        # 6
    "Golf ball"     # 7
]

# ====== 객체 카운트 ======
counter = Counter()

# txt 라벨 파일 순회
for txt_file in LABEL_DIR.glob("*.txt"):
    with open(txt_file, "r") as f:
        for line in f:
            parts = line.strip().split()
            if not parts:
                continue
            cls_id = int(parts[0])
            counter[cls_id] += 1

# ====== 결과 출력 ======
print("=== 클래스별 객체 개수 ===")
for i, cls_name in enumerate(classes):
    print(f"{cls_name:<15}: {counter.get(i, 0)}")

print("\n총 객체 수:", sum(counter.values()))

ValueError: invalid literal for int() with base 10: 'Total'

In [34]:
from pathlib import Path

# 대상 폴더
DATA_DIR = Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20250912_merge_data")

# 모든 .txt 파일 순회
deleted = 0
for txt in DATA_DIR.rglob("*.txt"):  # 하위 폴더까지 탐색
    if txt.stat().st_size == 0:      # 파일 크기가 0바이트 → 내용 없음
        txt.unlink()
        deleted += 1

print(f"[완료] 비어있는 txt 파일 {deleted}개 삭제")

[완료] 비어있는 txt 파일 242개 삭제


In [36]:
# === 8클래스 YOLO 데이터 분할(Train/Val/Test) + 리포트/복사/데이터셋 YAML 생성 ==========================
from pathlib import Path
from collections import Counter, defaultdict
import random, shutil

# ====== 설정 ======
SRC = Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20250912_merge_data")
DST = Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/2025092_merge_data_split")

# 이미지 확장자
IMG_EXTS = [".jpg", ".jpeg", ".png"]

# 분할 비율
RATIOS = {
    "train": 0.78,
    "val":   0.12,
    "test":  0.10,  # 필요 시 조정/0으로 비활성화 가능
}

# 클래스 ID (0~7)
# 0: Divot, 1: Fixed_Divot, 2: Diseased_Grass, 3: Confused_Object,
# 4: Pole, 5: Sprinkler, 6: Drain, 7: Golf ball
LABELS = list(range(8))

# 배경 표기
BG = "bg"

SEED = 42
random.seed(SEED)

# ====== 유틸 ======
def ensure_dirs():
    for s, r in RATIOS.items():
        if r <= 0:
            continue
        (DST / "images" / s).mkdir(parents=True, exist_ok=True)
        (DST / "labels" / s).mkdir(parents=True, exist_ok=True)

def find_image(stem: str):
    """SRC 직하 + 재귀에서 stem과 확장자가 일치하는 '첫 이미지' 반환 (파일명 유니크 가정)"""
    for ext in IMG_EXTS:
        p = SRC / f"{stem}{ext}"
        if p.exists():
            return p
    for ext in IMG_EXTS:
        hits = list(SRC.rglob(f"{stem}{ext}"))
        if hits:
            return hits[0]
    return None

def parse_label_file(p: Path):
    """해당 파일에 등장한 '클래스 집합'을 반환. 비어있으면 빈 set."""
    labs = set()
    try:
        with open(p, "r") as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) >= 5:
                    try:
                        cid = int(float(parts[0]))
                    except:
                        continue
                    if cid in LABELS:
                        labs.add(cid)
    except:
        pass
    return labs

# ====== 1) 수집: (stem, img_path, lbl_path or None, labels_set) ======
items = []
stems = set()

# 상위/하위 폴더 전체에서 이미지 수집
for ext in IMG_EXTS:
    for img in SRC.rglob(f"*{ext}"):
        stems.add(img.stem)

for stem in sorted(stems):
    img = find_image(stem)
    if img is None:
        continue
    lbl = img.with_suffix(".txt")
    if not lbl.exists():
        # 라벨 파일이 없으면 배경으로 취급
        items.append((stem, img, None, {BG}))
    else:
        labs = parse_label_file(lbl)
        if labs:
            items.append((stem, img, lbl, labs))
        else:
            # 빈 라벨 파일이면 배경으로 취급 (YOLO는 .txt 없으면 배경 처리 → 복사 시 라벨 생략)
            items.append((stem, img, None, {BG}))

if not items:
    raise SystemExit("No images found in source folder.")

all_labels = LABELS + [BG]
N = len(items)

# ====== 2) 목표치 계산 (샘플 개수 기준) ======
target_sizes = {s: int(round(N * r)) for s, r in RATIOS.items()}

# 합 보정(반올림 오차 보정)
while sum(target_sizes.values()) < N:
    k = min(target_sizes, key=lambda s: target_sizes[s])
    target_sizes[k] += 1
while sum(target_sizes.values()) > N:
    k = max(target_sizes, key=lambda s: target_sizes[s])
    target_sizes[k] -= 1

# 전체 라벨 분포 집계 (멀티라벨: 이미지 하나가 여러 라벨을 가질 수 있음)
global_counts = Counter()
for _, _, _, labs in items:
    for l in labs:
        global_counts[l] += 1

# 분할별 라벨 목표(비율 유지)
label_targets = {s: {l: global_counts[l]*RATIOS[s] for l in all_labels} for s in RATIOS}

# ====== 3) Iterative stratification ======
# 각 라벨이 포함된 샘플 인덱스 집합
idxs_by_label = {l: set() for l in all_labels}
for i, it in enumerate(items):
    for l in it[3]:
        idxs_by_label[l].add(i)

assigned = {s: [] for s in RATIOS}
split_sizes = {s: 0 for s in RATIOS}
split_label_counts = {s: Counter() for s in RATIOS}
unassigned = set(range(N))

def cap_ok(s):
    return split_sizes[s] < target_sizes[s]

def need(s, l):
    return label_targets[s][l] - split_label_counts[s][l]

def pick_label():
    # 남은 필요량이 큰 라벨 + 희소성 가중치 고려
    best_l, best_score = None, -1e18
    for l in all_labels:
        if not (idxs_by_label[l] & unassigned):
            continue
        pos_need_sum = sum(max(0.0, need(s, l)) for s in RATIOS if cap_ok(s))
        rarity_w = 1.0 / max(1.0, global_counts[l])  # 희소할수록 가중↑
        score = pos_need_sum * rarity_w
        if score > best_score:
            best_score, best_l = score, l
    return best_l

def assign_idx_to_best_split(i):
    labs = items[i][3]
    best_s, best_cost = None, 1e18
    for s in RATIOS:
        if not cap_ok(s):
            continue
        # 라벨 오차 변화 + 사이즈 편차(가벼운 가중치)
        label_term = 0.0
        for l in labs:
            cur = split_label_counts[s][l]
            tgt = label_targets[s][l]
            label_term += abs((cur+1) - tgt) - abs(cur - tgt)
        size_term = abs((split_sizes[s]+1) - target_sizes[s]) - abs(split_sizes[s] - target_sizes[s])
        cost = label_term + 0.05*size_term
        if cost < best_cost:
            best_cost, best_s = cost, s
    if best_s is None:
        return False
    assigned[best_s].append(i)
    split_sizes[best_s] += 1
    for l in labs:
        split_label_counts[best_s][l] += 1
    unassigned.discard(i)
    for l in labs:
        idxs_by_label[l].discard(i)
    return True

# 메인 루프
while unassigned:
    l = pick_label()
    if l is None:
        break
    pool = list(idxs_by_label[l] & unassigned)
    random.shuffle(pool)

    # l 라벨이 더 필요한 split 순으로 채우기
    splits_by_need = sorted([s for s in RATIOS if cap_ok(s)], key=lambda s: need(s, l), reverse=True)
    progressed = False
    for s in splits_by_need:
        best_i, best_cost = None, 1e18
        for i in pool:
            labs = items[i][3]
            label_term = 0.0
            for ll in labs:
                cur = split_label_counts[s][ll]
                tgt = label_targets[s][ll]
                label_term += abs((cur+1) - tgt) - abs(cur - tgt)
            size_term = abs((split_sizes[s]+1) - target_sizes[s]) - abs(split_sizes[s] - target_sizes[s])
            cost = label_term + 0.05*size_term
            if cost < best_cost:
                best_cost, best_i = cost, i
        if best_i is not None:
            assigned[s].append(best_i)
            split_sizes[s] += 1
            for ll in items[best_i][3]:
                split_label_counts[s][ll] += 1
            unassigned.discard(best_i)
            for ll in items[best_i][3]:
                idxs_by_label[ll].discard(best_i)
            pool.remove(best_i)
            progressed = True
            if not cap_ok(s):
                continue
    if not progressed:
        break

# 남은 샘플은 사이즈/라벨 오차 최소 split로 배정
if unassigned:
    rest = list(unassigned)
    random.shuffle(rest)
    for i in rest:
        assign_idx_to_best_split(i)

# ====== 4) 복사 ======
ensure_dirs()

def copy_pair(idx, split):
    stem, img, lbl, labs = items[idx]
    dst_img = DST / "images" / split / img.name
    dst_lbl = DST / "labels" / split / f"{stem}.txt"
    shutil.copy2(img, dst_img)
    if lbl and lbl.exists():
        shutil.copy2(lbl, dst_lbl)
    # bg는 라벨 파일 생성 안 함 (YOLO가 없는 경우를 배경으로 처리)

for s, idxs in assigned.items():
    for i in idxs:
        copy_pair(i, s)

# ====== 5) data.yaml (test 포함) ======
yaml_path = DST / "data.yaml"
with open(yaml_path, "w") as f:
    f.write("train: images/train\n")
    f.write("val: images/val\n")
    if target_sizes.get("test", 0) > 0:
        f.write("test: images/test\n")
    f.write("\n")
    f.write("nc: 8\n")
    f.write('names: ["Divot","Fixed_Divot","Diseased_Grass","Confused_Object","Pole","Sprinkler","Drain","Golf ball"]\n')

# ====== 6) 리포트 (train/val/test 모두) ======
def summarize_all():
    splits = list(RATIOS.keys())
    rep = []
    total = sum(len(assigned.get(s, [])) for s in splits)
    rep.append(f"Total images: {total}")
    rep.append(f"Targets: {target_sizes}")
    rep.append("Global label counts: " + ", ".join([f"{l}:{global_counts[l]}" for l in all_labels]))
    rep.append("Global label ratios: " + ", ".join([f"{l}:{global_counts[l]/max(1,total):.3f}" for l in all_labels]))
    for s in splits:
        if s not in assigned:
            continue
        n = len(assigned[s])
        cnt = split_label_counts.get(s, Counter())
        rep.append(f"\n[{s}] images: {n} (target {target_sizes.get(s, 0)})")
        rep.append("  label counts: " + ", ".join([f"{l}:{cnt[l]}" for l in all_labels]))
        rep.append("  label ratios: " + ", ".join([f"{l}:{cnt[l]/max(1,n):.3f}" for l in all_labels]))
        # 타깃 대비 라벨 분포 오차(참고용)
        errs = []
        for l in all_labels:
            tgt = label_targets[s][l]
            err = cnt[l] - tgt
            rel = 0.0 if tgt == 0 else err / max(1.0, tgt)
            errs.append(f"{l}:Δ{err:.1f}({rel:+.1%})")
        rep.append("  errors vs target: " + ", ".join(errs))
    return "\n".join(rep)

report = summarize_all()
print(report)
(DST / "split_report.txt").write_text(report, encoding="utf-8")

print(f"\n✅ Done. Split at: {DST}")
print(f"↳ Saved: {yaml_path}, split_report.txt")


Total images: 6650
Targets: {'train': 5187, 'val': 798, 'test': 665}
Global label counts: 0:4136, 1:3912, 2:653, 3:34, 4:366, 5:101, 6:425, 7:464, bg:363
Global label ratios: 0:0.622, 1:0.588, 2:0.098, 3:0.005, 4:0.055, 5:0.015, 6:0.064, 7:0.070, bg:0.055

[train] images: 5187 (target 5187)
  label counts: 0:3214, 1:2675, 2:510, 3:27, 4:285, 5:77, 6:301, 7:362, bg:268
  label ratios: 0:0.620, 1:0.516, 2:0.098, 3:0.005, 4:0.055, 5:0.015, 6:0.058, 7:0.070, bg:0.052
  errors vs target: 0:Δ-12.1(-0.4%), 1:Δ-376.4(-12.3%), 2:Δ0.7(+0.1%), 3:Δ0.5(+1.8%), 4:Δ-0.5(-0.2%), 5:Δ-1.8(-2.3%), 6:Δ-30.5(-9.2%), 7:Δ0.1(+0.0%), bg:Δ-15.1(-5.3%)

[val] images: 798 (target 798)
  label counts: 0:508, 1:666, 2:78, 3:4, 4:44, 5:12, 6:70, 7:56, bg:52
  label ratios: 0:0.637, 1:0.835, 2:0.098, 3:0.005, 4:0.055, 5:0.015, 6:0.088, 7:0.070, bg:0.065
  errors vs target: 0:Δ11.7(+2.4%), 1:Δ196.6(+41.9%), 2:Δ-0.4(-0.5%), 3:Δ-0.1(-2.0%), 4:Δ0.1(+0.2%), 5:Δ-0.1(-1.0%), 6:Δ19.0(+37.3%), 7:Δ0.3(+0.6%), bg:Δ8.4(+19.4%)


In [ ]:
from ultralytics import YOLO

# 데이터 설정: train/val 경로, 클래스 수/이름이 정확해야 함
DATA_YAML = "/home/dw/ws_job_msislab/Golf_Project/data/for_study/20250912_merge_data_split/data.yaml"

# 모델 선택: s(가볍고 빠름) → 결과 좋아지면 m으로 확장
model = YOLO("yolov8s.pt")  # ← 여기만 바뀌면 됩니다

results = model.train(
    # === 기본 학습 설정 ===
    data=DATA_YAML,      # 학습/검증 경로 정의된 data.yaml
    epochs=800,          # 총 학습 epoch (조기종료가 있으면 실제로는 더 적게 끝날 수 있음)
    patience=50,         # val metric이 개선 없을 때 조기종료 대기 epoch
    imgsz=1024,          # 입력 해상도(정사각). 소물체 ↑. OOM 시 896/768로
    batch=-1,            # 자동 배치 크기 탐색. OOM 나면 8~16으로 수동 고정
    device=0,            # GPU 인덱스 (여러 GPU면 "0,1" 형태 가능)
    workers=8,           # DataLoader 병렬 로드 워커 수 (디스크/CPU 상황에 맞춰)
    rect=True,           # aspect ratio 유지한 사각형 배치 → 소물체 왜곡↓, 수렴 안정화
    cache=True,          # 이미지/라벨 메모리 캐시. RAM 부족 시 False 권장
    amp=True,            # 자동 혼합정밀도(fp16). 최신 GPU에서 속도/메모리 이점

    # === 온라인 증강(가볍게) ===
    mosaic=0.10,         # 초반 모자이크 비율. 소물체 리콜↑. 과하면 artifacts↑
    close_mosaic=15,     # 지정 epoch 이후 mosaic=0 자동 전환(과적합 방지)
    mixup=0.0,           # MixUp 비활성(디봇/잔디 경계 왜곡 방지 목적)
    copy_paste=0.0,      # Copy-Paste 비활성(객체 경계/텍스처 왜곡 우려)
    hsv_h=0.01,          # Hue 변동(작게). 잔디 색 왜곡 최소화
    hsv_s=0.2,           # Saturation 변동
    hsv_v=0.2,           # Value(밝기) 변동
    degrees=0.0,         # 회전 금지(지형/그림자 방향성 유지 목적)
    translate=0.0,       # 이동 변형 금지(필요 시 0.05~0.1 권장)
    scale=0.0,           # 스케일 변형 금지(필요 시 0.10~0.20 권장)
    shear=0.0,           # 기울이기 금지(텍스처 왜곡 방지)
    erasing=0.0,         # 랜덤 Erasing 비활성(작은 객체 소실 방지)
    # ※ 오프라인 증강을 이미 다수 수행했다면 온라인은 약하게 유지하는 편이 안전

    # === 손실/최적화 쪽 튜닝 ===
    box=10.0,            # bbox 회귀 손실 가중(기본≈7.5). 위치 정밀도↑
    cls=0.5,             # 클래스 분류 손실 가중(기본 수준)
    dfl=1.5,             # Distribution Focal Loss 가중(경계 정밀도↑)
    # lr0/lrf: Cosine 스케줄(초기 lr → 최종 lr=lr0*lrf)
    lr0=0.002,           # 초기 lr (AdamW 기준 안전값 / SGD면 보통 0.01)
    lrf=0.01,            # 최종 lr 비율 (0.01이면 마지막에 lr0의 1%)
    momentum=0.9,        # SGD: 모멘텀, Adam/AdamW: beta1 역할로 사용됨(ultra 내부 호환)
    weight_decay=0.01,   # AdamW 권장: 0.01 / SGD 권장: 0.0005

    # === Optimizer 선택 ===
    # optimizer 인자를 생략하면 기본 SGD로 동작
    # optimizer="SGD",     # 대규모 데이터/보수적 일반화. 권장 세팅: lr0=0.01, wd=0.0005, momentum=0.937
    optimizer="AdamW",    # 소/중규모+증강 多에서 빠르고 안정적 수렴. 권장: lr0≈0.002, wd≈0.01
    # optimizer="Adam",    # 빠른 수렴(초반 성능↑)이나 과적합 주의. 권장: lr0≈0.001~0.002, wd 0~0.0005
    # optimizer="RMSProp", # 특수 케이스 외 권장X. 튜닝 민감.
    # optimizer="NAdam",   # Adam+Nesterov. 드물게 사용. 실험용.

    # === 출력 경로 ===
    project="/home/dw/ws_job_msislab/Golf_Project/runs_yolo",  # 상위 폴더
    name="20250911_merge_data_yolov8m",  # 실험 이름(여러 실험 구분용)
    exist_ok=True            # 같은 이름 덮어쓰기 허용
)

Ultralytics 8.3.175 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 7932MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=10.0, cache=True, cfg=None, classes=None, close_mosaic=15, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/dw/ws_job_msislab/Golf_Project/data/for_study/20250911_merge_data_split/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=500, erasing=0.0, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.01, hsv_s=0.2, hsv_v=0.2, imgsz=1024, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.002, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.9, mosaic=0.1, multi_scale=False, name=20250911_merge_data_yolov8m, nbs=64, nms=False, opset=None, optimize=False, optim

train: Scanning /home/dw/ws_job_msislab/Golf_Project/data/for_study/20250911_merge_data_split/labels/train.cache... 1562 images, 224 backgrounds, 0 corrupt: 100%|██████████| 1786/1786 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
AutoBatch: Computing optimal batch size for imgsz=1024 at 60.0% CUDA memory utilization.
AutoBatch: CUDA:0 (NVIDIA GeForce RTX 4060 Ti) 7.75G total, 0.24G reserved, 0.24G allocated, 7.27G free
      Params      GFLOPs  GPU_mem (GB)  forward (ms) backward (ms)                   input                  output
    25860952       202.5         2.338         24.43         151.6      (1, 3, 1024, 1024)                    list
    25860952       404.9         3.674         32.72         74.38      (2, 3, 1024, 1024)                    list
    25860952       809.9         5.721         66.11         129.3      (4, 3, 1024, 1024)                    list
CUDA out of memory. Tried to allocate 48.00 MiB. GPU 0 has a total capacity of 7.75 GiB of which 26.06 MiB is free. Including non-PyT

train: Scanning /home/dw/ws_job_msislab/Golf_Project/data/for_study/20250911_merge_data_split/labels/train.cache... 1562 images, 224 backgrounds, 0 corrupt: 100%|██████████| 1786/1786 [00:00<?, ?it/s]


WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    285/500      2.31G     0.3345     0.1916     0.7276          4       1024: 100%|██████████| 893/893 [01:17<00:00, 11.59it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 76/76 [00:04<00:00, 17.24it/s]


                   all        301        784      0.726      0.645      0.697      0.532
EarlyStopping: Training stopped early as no improvement observed in last 50 epochs. Best results observed at epoch 235, best model saved as best.pt.
To update EarlyStopping(patience=50) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

285 epochs completed in 6.477 hours.
Optimizer stripped from /home/dw/ws_job_msislab/Golf_Project/runs_yolo/20250911_merge_data_yolov8m/weights/last.pt, 52.1MB
Optimizer stripped from /home/dw/ws_job_msislab/Golf_Project/runs_yolo/20250911_merge_data_yolov8m/weights/best.pt, 52.1MB

Validating /home/dw/ws_job_msislab/Golf_Project/runs_yolo/20250911_merge_data_yolov8m/weights/best.pt...
Ultralytics 8.3.175 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 7932MiB)
Model summary (fused): 92 layers, 25,844,392 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 76/76 [00:03<00:00, 19.18it/s]


                   all        301        784        0.8      0.706      0.754      0.557
                 Divot        184        342      0.926      0.771      0.877      0.698
           Fixed_Divot        147        271      0.802      0.806      0.836      0.676
        Diseased_Grass         46         74      0.389      0.351      0.304      0.249
                  Pole         25         25      0.796        0.8       0.86      0.353
             Sprinkler          7         17      0.766      0.387      0.412      0.306
                 Drain         25         25      0.923          1      0.995      0.839
             Golf ball         30         30          1      0.825      0.995       0.78
Speed: 0.4ms preprocess, 11.4ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to /home/dw/ws_job_msislab/Golf_Project/runs_yolo/20250911_merge_data_yolov8m
